In [ ]:
# 5_model_optimization_v2.ipynb
# Optimized Spatiotemporal Model Training
# Architecture: ST-ConvNet V2 (Sequential Encoder + Asymmetric Schedule Processing)
# Scaling: RobustScaler (Median/IQR)

import os
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import joblib # For saving the scaler

# --- Environment Setup (Colab vs Local) ---
try:
    from google.colab import drive
    print("Detected Colab Environment. Mounting Drive...")
    drive.mount('/content/drive')
    
    # UPDATE THIS PATH to match your Google Drive folder structure
    PROJECT_ROOT = "/content/drive/MyDrive/headway-prediction"
    
    # Add project root to system path so we can import src.config, src.models
    sys.path.append(PROJECT_ROOT)
    
except ImportError:
    print("Detected Local Environment.")
    # Assumes notebook is in 'notebooks/' and project root is one level up
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
    sys.path.append(PROJECT_ROOT)

print(f"Project Root: {PROJECT_ROOT}")

# Validate Imports from src (sanity check)
try:
    from src.config import Config
    from src.models.st_covnet_v2 import HeadwayConvLSTM_V2
    print("✅ Successfully imported 'src' modules.")
except ImportError as e:
    print(f"❌ Failed to import 'src' modules: {e}")
    print("Please ensure PROJECT_ROOT points to the folder containing 'src/'")

# Paths
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
print(f"Data Directory: {DATA_DIR}")